# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from IPython.display import display
%matplotlib inline
pd.set_option('display.notebook_repr_html', True)

In [2]:
os.getcwd()

'C:\\Users\\Maribel'

In [3]:
path=r"E:\Maribel\Documents\Data Science Springboard\Unit 6\6.4"
os.chdir(path)

os.getcwd()

os.listdir()

['GuidedCapstoneStep4 and step5HL.ipynb',
 'GuidedCapstoneStep4wmastercsv.ipynb',
 'Notebook_stepfour.ipynb',
 'step3FULL_output.csv',
 'step3_output.csv']

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [4]:
file = r'E:\Maribel\Documents\Data Science Springboard\Unit 6\6.4\step3FULL_output.csv'
df4 = pd.read_csv(file, index_col=0)
print(df4.shape)
df4.head()

(330, 27)


,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,250,1,0.0,0,2,2,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,2
1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,2
2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,2
3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0.0,1,0,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [5]:
df4.dtypes

Name                  object
state                 object
summit_elev            int64
vertical_drop          int64
base_elev              int64
trams                  int64
fastEight            float64
fastSixes              int64
fastQuads              int64
quad                   int64
triple                 int64
double                 int64
surface                int64
total_chairs           int64
Runs                 float64
TerrainParks         float64
LongestRun_mi        float64
SkiableTerrain_ac    float64
Snow Making_ac       float64
daysOpenLastYear     float64
yearsOpen            float64
averageSnowfall      float64
AdultWeekday         float64
AdultWeekend         float64
projectedDaysOpen    float64
NightSkiing_ac       float64
clusters               int64
dtype: object

In [6]:
df4 = pd.concat([df4, pd.get_dummies(df4['state'])], axis=1).drop(['state'], axis =1)
print(df4.shape)
df4.head()

(330, 61)


,Name,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Alyeska Resort,3939,2500,250,1,0.0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Eaglecrest Ski Area,2600,1540,1200,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hilltop Ski Area,2090,294,1796,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Arizona Snowbowl,11500,2300,9200,0,0.0,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,11100,1800,9200,0,0.0,0,1,2,3,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [7]:
# first we import the preprocessing package from the sklearn library
df = df4
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X
# What does it mean for stanrdization where each feature value will have the mean subtracted and divided by the SD?
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X
# Is this used to impute the data(like Na'ans)? Why not use fit_transform() instead?
X_scaled=scaler.transform(X) 

In [8]:
# This is the explanatory variable(s) that are used to predict the response variable.
X

,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,3939,2500,250,1,0.0,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2600,1540,1200,0,0.0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
2,2090,294,1796,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,11500,2300,9200,0,0.0,1,0,2,2,1,...,0,0,0,0,0,0,0,0,0,0
4,11100,1800,9200,0,0.0,0,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,9500,1000,8500,0,0.0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
326,7428,810,6619,0,0.0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
327,7808,1571,6237,0,0.0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
328,9663,990,8798,0,0.0,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,1


In [9]:
# The response variable that whose variation depends on the other variables. 
y

0      85.00000
1      53.00000
2      34.00000
3      89.00000
4      78.00000
         ...   
325    64.16681
326    42.00000
327    59.00000
328    49.00000
329    49.00000
Name: AdultWeekend, Length: 330, dtype: float64

In [10]:
X_scaled

array([[-0.1750243 ,  1.3572862 , -1.0037286 , ..., -0.11076976,
        -0.22573306, -0.15762208],
       [-0.53401788,  0.34294523, -0.69849743, ..., -0.11076976,
        -0.22573306, -0.15762208],
       [-0.6707518 , -0.97358483, -0.50700504, ..., -0.11076976,
        -0.22573306, -0.15762208],
       ...,
       [ 0.86227679,  0.37569999,  0.91987035, ..., -0.11076976,
        -0.22573306,  6.34428877],
       [ 1.35961292, -0.23818762,  1.74270932, ..., -0.11076976,
        -0.22573306,  6.34428877],
       [ 1.31591169, -0.12196105,  1.61483353, ..., -0.11076976,
        -0.22573306,  6.34428877]])

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [11]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
# What does random_state do and why did we do it?
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [12]:
#What is the difference between this and the y above?
y

array([ 85.        ,  53.        ,  34.        ,  89.        ,
        78.        ,  47.        ,  64.16681004,  64.16681004,
        64.16681004,  78.        ,  75.        ,  64.16681004,
        64.16681004,  64.16681004, 159.        ,  59.        ,
        84.        ,  69.        ,  64.16681004,  64.16681004,
        83.        ,  64.16681004,  79.        ,  50.        ,
       125.        ,  69.        ,  85.        , 179.        ,
        64.16681004,  64.16681004, 158.        ,  64.16681004,
        89.        ,  64.16681004,  25.        ,  64.16681004,
        79.        ,  89.        ,  71.        ,  79.        ,
        56.        ,  84.        ,  64.16681004,  65.        ,
       139.        ,  64.16681004,  64.16681004,  72.        ,
        65.        ,  60.        ,  55.        ,  62.        ,
        42.        ,  64.        ,  70.        ,  42.        ,
        47.        ,  32.        ,  47.        ,  64.16681004,
        81.        ,  62.        ,  43.        ,  64.16

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [13]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [14]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [11]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [15]:
explained_variance_score(y_test, y_pred)

0.9320408073936114

In [16]:
mean_absolute_error(y_test, y_pred)

5.150032097510979

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [17]:
lm.intercept_

64.11941937993103

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [15]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
# pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
coeff_df = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
New York,1.539818e+12
Michigan,1.453170e+12
Colorado,1.280327e+12
California,1.252919e+12
total_chairs,1.224535e+12
Pennsylvania,1.195615e+12
New Hampshire,1.102450e+12
Wisconsin,1.102450e+12
Vermont,1.069141e+12
Minnesota,1.034527e+12


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [19]:
file = r'E:\Maribel\Documents\Data Science Springboard\Unit 6\6.4\step3FULL_output.csv'
df = pd.read_csv(file, index_col=0)
print(df.shape)
X=df.drop(['AdultWeekend','Name','state'], axis=1)
y=df.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 
y=y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
y_pred = model.predict(X_test)
mean_absolute_error(y_test, y_pred)
lm.intercept_
print("Explained Variance Score: ", explained_variance_score(y_test, y_pred))
print("Mean Absolute Error: ", mean_absolute_error(y_test, y_pred) )
print("lm.intercept: ", lm.intercept_)

(330, 27)
Explained Variance Score:  0.9234484111167843
Mean Absolute Error:  5.554207308676172
lm.intercept:  64.08326717241651


In [20]:
coeff_df = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
AdultWeekday,19.897868
summit_elev,14.029309
base_elev,11.902176
vertical_drop,5.174457
quad,1.567309
Runs,1.533770
averageSnowfall,1.514134
triple,1.416570
surface,1.331088
clusters,1.307734


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [21]:
file = r'E:\Maribel\Documents\Data Science Springboard\Unit 6\6.4\step3FULL_output.csv'
df = pd.read_csv(file, index_col=0)
print(df.shape)
X=df.drop(['Name','AdultWeekend','state','summit_elev','base_elev'], axis=1)
y=df.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 
y=y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
y_pred = model.predict(X_test)
mean_absolute_error(y_test, y_pred)
lm.intercept_
print("Explained Variance Score: ", explained_variance_score(y_test, y_pred))
print("Mean Absolute Error: ", mean_absolute_error(y_test, y_pred) )
print("lm.intercept: ", lm.intercept_)

(330, 27)
Explained Variance Score:  0.929979384247776
Mean Absolute Error:  5.356055528318803
lm.intercept:  64.08639675870683


In [19]:
coeff_df = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
AdultWeekday,19.877089
vertical_drop,1.806536
Runs,1.645646
quad,1.516478
averageSnowfall,1.422438
triple,1.401407
surface,1.263226
clusters,1.259251
daysOpenLastYear,0.896706
fastQuads,0.745562


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.9320408073936114 | 5.150032097510979 |-|
| Model 2. | 0.9234484111167843| 5.554207308676172 |'state'|
| Model 3. | 0.929979384247776 | 5.356055528318803 |'state','summit_elev','base_elev'|

Model Selection: Model 3. 